In [ ]:

'''
all_files = [
    '107f24d6e9_F1BE1D4184INSPIRE', '11cdce7802_B6A62F8BE0INSPIRE', '12fa5e614f_53197F206FOPENPIPELINE', '130a76ebe1_68B40B480AOPENPIPELINE', 
    '1476907971_CHADGRISMOPENPIPELINE', '1553541487_APIGENERATED', '1553541585_APIGENERATED', '1553627230_APIGENERATED', '15efe45820_D95DF0B1F4INSPIRE', 
    '1726eb08ef_60693DB04DINSPIRE', '1d056881e8_29FEA32BC7INSPIRE', '1d4fbe33f3_F1BE1D4184INSPIRE', '1df70e7340_4413A67E91INSPIRE', '2552eb56dd_2AABB46C86OPENPIPELINE', 
    '25f1c24f30_EB81FE6E2BOPENPIPELINE', '2ef3a4994a_0CCD105428INSPIRE', '2ef883f08d_F317F9C1DFOPENPIPELINE', '34fbf7c2bd_E8AD935CEDINSPIRE', 
    '3502e187b2_23071E4605OPENPIPELINE', '39e77bedd0_729FB913CDOPENPIPELINE', '420d6b69b8_84B52814D2OPENPIPELINE', '520947aa07_8FCB044F58OPENPIPELINE', 
    '551063e3c5_8FCB044F58INSPIRE', '57426ebe1e_84B52814D2OPENPIPELINE', '5fa39d6378_DB9FF730D9OPENPIPELINE', '6f93b9026b_F1BFB8B17DOPENPIPELINE', 
    '7008b80b00_FF24A4975DINSPIRE', '74d7796531_EB81FE6E2BOPENPIPELINE', '7c719dfcc0_310490364FINSPIRE', '84410645db_8D20F02042OPENPIPELINE', 
    '8710b98ea0_06E6522D6DINSPIRE', '888432f840_80E7FD39EBINSPIRE', '9170479165_625EDFBAB6OPENPIPELINE', 'a1af86939f_F1BE1D4184OPENPIPELINE', 
    'b61673f780_4413A67E91INSPIRE', 'b705d0cc9c_E5F5E0E316OPENPIPELINE', 'b771104de5_7E02A41EBEOPENPIPELINE', 'c2e8370ca3_3340CAC7AEOPENPIPELINE', 
    'c37dbfae2f_84B52814D2OPENPIPELINE', 'c644f91210_27E21B7F30OPENPIPELINE', 'c6d131e346_536DE05ED2OPENPIPELINE', 'c8a7031e5f_32156F5DC2INSPIRE', 
    'cc4b443c7d_A9CBEF2C97INSPIRE', 'd06b2c67d2_2A62B67B52OPENPIPELINE', 'd9161f7e18_C05BA1BC72OPENPIPELINE', 'dabec5e872_E8AD935CEDINSPIRE', 
    'e87da4ebdb_29FEA32BC7INSPIRE', 'ebffe540d0_7BA042D858OPENPIPELINE', 'ec09336a6f_06BA0AF311OPENPIPELINE', 
    'f0747ed88d_E74C0DD8FDOPENPIPELINE', 'f4dd768188_NOLANOPENPIPELINE', 'f56b6b2232_2A62B67B52OPENPIPELINE', 
    'f971256246_MIKEINSPIRE', 'f9f43e5144_1DB9E6F68BINSPIRE', 'fc5837dcf8_7CD52BE09EINSPIRE'
]
'''

val_files = [
    "a1af86939f_F1BE1D4184OPENPIPELINE",
    "c644f91210_27E21B7F30OPENPIPELINE",
    "f9f43e5144_1DB9E6F68BINSPIRE",
    "1d056881e8_29FEA32BC7INSPIRE",
    "3502e187b2_23071E4605OPENPIPELINE",
    "d9161f7e18_C05BA1BC72OPENPIPELINE",
    "c8a7031e5f_32156F5DC2INSPIRE",
    "551063e3c5_8FCB044F58INSPIRE",
    "fc5837dcf8_7CD52BE09EINSPIRE",
    "39e77bedd0_729FB913CDOPENPIPELINE",
]

test_files = [
    "25f1c24f30_EB81FE6E2BOPENPIPELINE",
    "1d4fbe33f3_F1BE1D4184INSPIRE",
    "15efe45820_D95DF0B1F4INSPIRE",
    "107f24d6e9_F1BE1D4184INSPIRE",
    "c6d131e346_536DE05ED2OPENPIPELINE",
    "12fa5e614f_53197F206FOPENPIPELINE",
    "5fa39d6378_DB9FF730D9OPENPIPELINE",
    "ebffe540d0_7BA042D858OPENPIPELINE",
    "8710b98ea0_06E6522D6DINSPIRE",
    "84410645db_8D20F02042OPENPIPELINE",
]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_path = "/content/chipped_data/content/chipped_data/train_metadata.csv"
raw_path = "/content/chipped_data/content/chipped_data/raw/raw_metadata.csv"
#train_path = "train_metadata.csv"
#raw_path = "raw_metadata.csv"


NUM_CLASSES = 6
class_cols = ['0: Building', '1: Clutter', '2: Vegetation', '3: Water', '4: Background', '5: Car']
CLASS_NAMES = ['Building', 'Clutter', 'Vegetation', 'Water', 'Background', 'Car']
COLOR_TO_CLASS = {
    (230, 25, 75): 0,
    (145, 30, 180): 1,
    (60, 180, 75): 2,
    (245, 130, 48): 3,
    (255, 255, 255): 4,
    (0, 130, 200): 5,
    (255, 0, 255): 6 # Ignore pixel for visualisation
}
CLASS_TO_COLOR = {v: k for k, v in COLOR_TO_CLASS.items() if v < 6} # Exclude ignore color

def csv_to_df(split, chip_size, ignore_threshold=0.95):
    """
    Returns a DataFrame filtered by split with optional ignore pixel filtering for val/test.
    
    Args:
        split (str): One of 'train', 'val', or 'test'
        chip_size (int): Size of the chips, either 1024 or 256
        ignore_threshold (float): Maximum allowed ignore pixel ratio for val/test

    Returns:
        pd.DataFrame: Filtered DataFrame for the requested split
    """

    if chip_size == 1024:
        metadata_path = "/content/chipped_data/content/chipped_data/raw/raw_metadata.csv"
    elif chip_size == 256:
        metadata_path = "/content/chipped_data/content/chipped_data/train_metadata.csv"
    else:
        raise ValueError("Invalid chip_size. Use 1024 or 256.")


    if split == 'train':
        df = pd.read_csv(metadata_path)
        excluded_prefixes = val_files + test_files
        is_excluded = df['tile_id'].apply(lambda tid: any(tid.startswith(prefix) for prefix in excluded_prefixes))
        train_df = df[~is_excluded].copy()

        # Class info
        CLASS_NAMES = ['Building', 'Clutter', 'Vegetation', 'Water', 'Background', 'Car']
        class_cols = [f'{i}: {name}' for i, name in enumerate(CLASS_NAMES)]

        # Target class distribution
        target_subset = 0.15
        targets = {
            '0: Building': 0.16,
            '1: Clutter': 0.004,
            '2: Vegetation': 0.09,
            '3: Water': 0.4,
            '4: Background': None,
            '5: Car': 0.3
        }
        targets['4: Background'] = 1 - sum(v for v in targets.values() if v is not None)

        '''
        target_subset = 0.5
        targets = {
            '0: Building': 0.21,
            '1: Clutter': 0.1,
            '2: Vegetation': 0.05,
            '3: Water': 0.33,
            '4: Background': None,
            '5: Car': 0.33
        }
        '''

        # Normalize per-chip class distributions
        train_df['total'] = train_df[class_cols].sum(axis=1)
        for col in class_cols:
            train_df[col + '_norm'] = train_df[col] / train_df['total']

        # Score chips by how close they are to target (L1 distance)
        def compute_score(row):
            return sum(abs(row[col + '_norm'] - tgt) for col, tgt in targets.items())

        train_df['score'] = train_df.apply(compute_score, axis=1)

        # Select top-ranked chips
        n_chips = int(len(train_df) * (target_subset))
        train_df_sorted = train_df.sort_values('score').head(n_chips)

        # Distribution analysis
        subset = train_df_sorted[class_cols].sum()
        actual_distribution = subset / subset.sum()

        summary_df = pd.DataFrame({
            'Class': [col.split(":")[1].strip() for col in class_cols],
            'Target': [targets[col] for col in class_cols],
            'Actual': actual_distribution.values
        })
        summary_df['Difference'] = summary_df['Actual'] - summary_df['Target']

        from IPython.display import display
        display(summary_df)

        plot_class_distribution_from_df(train_df_sorted, title="Training Class Distribution")

        actual_chip_count = len(train_df_sorted)
        original_chip_count = len(train_df)
        actual_fraction = actual_chip_count / original_chip_count
        print(f"\n📦 Selected {actual_chip_count:,} chips from {original_chip_count:,} total "
            f"({actual_fraction:.2%} of available training chips)")

        return train_df_sorted



    elif split in ['val', 'test']:
        df = pd.read_csv(metadata_path)
        file_list = val_files if split == 'val' else test_files

        # Filter by matching tile IDs
        df = df[df['tile_id'].apply(lambda tid: any(tid.startswith(p) for p in file_list))].copy()

        if chip_size == 256:
            df = df[(df['x'] % 256 == 0) & (df['y'] % 256 == 0)].copy()

        if chip_size == 1024:
            # Sum class columns 0: to 5: to estimate non-ignore content
            class_cols = [col for col in df.columns if any(col.startswith(f"{i}:") for i in range(6))]
            df['non_ignore_sum'] = df[class_cols].sum(axis=1)

            required_non_ignore = 1.0 - ignore_threshold
            df = df[df['non_ignore_sum'] >= required_non_ignore].copy()

        return df

    else:
        raise ValueError(f"Invalid split: {split}. Choose from 'train', 'val', 'test'.")




# --- Function ---
def plot_class_distribution_from_df(dataframe, title="Training Class Distribution"):
    pixel_sums = dataframe[class_cols].sum()
    pixel_props = pixel_sums / pixel_sums.sum()

    class_labels = [f"{i}: {CLASS_NAMES[i]}" for i in range(NUM_CLASSES)]
    colours = [np.array(CLASS_TO_COLOR[i]) / 255.0 for i in range(NUM_CLASSES)]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_labels, pixel_props, color=colours, edgecolor='black')
    plt.title(title)
    plt.xlabel("Class")
    plt.ylabel("Proportion")
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)

    for bar, prop in zip(bars, pixel_props):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{prop:.2%}",
                 ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()



,tile_id,source_file,x,y,0: Building,1: Clutter,2: Vegetation,3: Water,4: Background,5: Car,entropy,non_ignore_sum
54,c6d131e346_536DE05ED2OPENPIPELINE_375_88,c6d131e346_536DE05ED2OPENPIPELINE,375,88,0.000000,0.003759,0.050560,0.000000,0.945680,0.000000,0.324187,1.0
55,c6d131e346_536DE05ED2OPENPIPELINE_375_1112,c6d131e346_536DE05ED2OPENPIPELINE,375,1112,0.000000,0.030376,0.000000,0.000000,0.969624,0.000000,0.196276,1.0
56,c6d131e346_536DE05ED2OPENPIPELINE_375_2136,c6d131e346_536DE05ED2OPENPIPELINE,375,2136,0.000000,0.017442,0.015028,0.000000,0.967530,0.000000,0.238972,1.0
57,c6d131e346_536DE05ED2OPENPIPELINE_1399_88,c6d131e346_536DE05ED2OPENPIPELINE,1399,88,0.004811,0.012634,0.070594,0.000000,0.910296,0.001665,0.525490,1.0
58,c6d131e346_536DE05ED2OPENPIPELINE_1399_1112,c6d131e346_536DE05ED2OPENPIPELINE,1399,1112,0.001460,0.065447,0.035162,0.000000,0.897931,0.000000,0.580487,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1975,9170479165_625EDFBAB6OPENPIPELINE_1208_1303,9170479165_625EDFBAB6OPENPIPELINE,1208,1303,0.050598,0.281437,0.000000,0.000000,0.666519,0.001446,1.136335,1.0
1976,9170479165_625EDFBAB6OPENPIPELINE_1208_2327,9170479165_625EDFBAB6OPENPIPELINE,1208,2327,0.000000,0.000471,0.041885,0.084936,0.872707,0.000000,0.670517,1.0
1977,9170479165_625EDFBAB6OPENPIPELINE_2232_279,9170479165_625EDFBAB6OPENPIPELINE,2232,279,0.000000,0.208069,0.000000,0.000000,0.788406,0.003525,0.770384,1.0
1978,9170479165_625EDFBAB6OPENPIPELINE_2232_1303,9170479165_625EDFBAB6OPENPIPELINE,2232,1303,0.149681,0.007206,0.000000,0.000000,0.827965,0.015148,0.778483,1.0
